## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-09-09-00-00 +0000,cbc,An era of 'wrecking ball' politics: What the M...,https://www.cbc.ca/news/politics/munich-securi...
1,2026-02-09-09-00-00 +0000,cbc,Cubans wonder if they could be next after Vene...,https://www.cbc.ca/news/politics/cuban-cia-cas...
2,2026-02-09-08-59-23 +0000,nyt,"Venezuela Frees Key Opposition Figures, Then R...",https://www.nytimes.com/2026/02/08/world/ameri...
3,2026-02-09-08-57-00 +0000,wsj,Taiwan Exports Start 2026 Strong in Bright Sig...,https://www.wsj.com/economy/trade/taiwan-expor...
4,2026-02-09-08-56-00 +0000,wsj,Magnitude of Takaichi Win Could Lead to Higher...,https://www.wsj.com/finance/currencies/yen-mos...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2546/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
40,super,24
45,bowl,23
106,election,18
117,trump,17
265,bad,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
71,2026-02-09-02-55-54 +0000,nypost,Trump slams Bad Bunny’s Super Bowl halftime sh...,https://nypost.com/2026/02/08/us-news/trump-sl...,123
67,2026-02-09-03-08-19 +0000,nyt,"Lady Gaga, Ricky Martin and Cardi B Join Bad B...",https://www.nytimes.com/2026/02/08/us/bad-bunn...,109
55,2026-02-09-04-00-40 +0000,latimes,"Bad Bunny fit in an actual wedding, with cake,...",https://www.latimes.com/california/story/2026-...,105
37,2026-02-09-05-42-38 +0000,nyt,Why the N.F.L. Booked Bad Bunny for the Super ...,https://www.nytimes.com/2026/02/08/business/ba...,101
51,2026-02-09-04-47-18 +0000,nyt,Bad Bunny Delivers a Love Letter to Puerto Ric...,https://www.nytimes.com/2026/02/08/arts/music/...,100


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
71,123,2026-02-09-02-55-54 +0000,nypost,Trump slams Bad Bunny’s Super Bowl halftime sh...,https://nypost.com/2026/02/08/us-news/trump-sl...
111,66,2026-02-08-23-43-59 +0000,bbc,Japan's PM Takaichi on course to win snap elec...,https://www.bbc.com/news/articles/cx2y7d2z29xo...
73,43,2026-02-09-02-29-00 +0000,wsj,"Hong Kong’s Jimmy Lai Given 20-Year Sentence, ...",https://www.wsj.com/world/china/hong-kongs-jim...
173,39,2026-02-08-18-04-45 +0000,nypost,Iran sentences locked-up Nobel Peace Prize win...,https://nypost.com/2026/02/08/world-news/iran-...
211,37,2026-02-08-12-00-00 +0000,wsj,The Trump economy in 2025 was a roller coaster...,https://www.wsj.com/economy/consumers/american...
17,30,2026-02-09-07-53-06 +0000,bbc,Venezuela's opposition says party leader kidna...,https://www.bbc.com/news/articles/c9wnjqzrv48o...
162,30,2026-02-08-19-11-00 +0000,wsj,U.K. Prime Minister Starmer’s Top Aide Resigns...,https://www.wsj.com/world/uk/u-k-prime-ministe...
120,30,2026-02-08-23-01-00 +0000,startribune,Lindsey Vonn breaks her left leg in Olympic do...,https://www.startribune.com/2026-winter-olympi...
160,29,2026-02-08-19-17-15 +0000,nypost,London’s Camden Goods Yard poised to draw New ...,https://nypost.com/2026/02/08/business/londons...
104,26,2026-02-09-00-16-25 +0000,nypost,Kroger to name former Walmart exec Greg Foran ...,https://nypost.com/2026/02/08/business/kroger-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
